# Trabalho Pŕatico 1 - Computação Gráfica
Feito por Gustavo Leão e Luigi Eliabe

In [30]:
class Window:
    def __init__(self,minX,minY,maxX,maxY) -> None:
        self.minX = minX
        self.minY = minY
        self.maxX = maxX
        self.maxY = maxY
        pass

    def __str__(self) -> str:
        
        return (f"({self.minX},{self.minY},{self.maxX},{self.maxY})")

In [31]:
class Viewport:
    def __init__(self,minX,minY,maxX,maxY) -> None:
        self.minX = minX
        self.minY = minY
        self.maxX = maxX
        self.maxY = maxY
        pass

    def __str__(self) -> str:
        
        return (f"({self.minX},{self.minY},{self.maxX},{self.maxY})")

In [32]:
class Ponto:
    def __init__(self,x,y) -> None:
        self.x = x
        self.y = y
        pass

    def __str__(self) -> str:
        return (f"({self.x},{self.y})")


In [33]:
class Reta:
    def __init__(self,*pontos) -> None:
        self.pontosReta = []
        for ponto in pontos:
            self.pontosReta.append(ponto)
        pass

    def __str__(self) -> str:
        return '('+','.join(str(e) for e in self.pontosReta)+')'

In [34]:
class Poligono:

    def __init__(self,*pontos) -> None:
        self.pontosPoligono = []
        for ponto in pontos:
            self.pontosPoligono.append(ponto)
        pass

    
    def __str__(self) -> str:
        return '('+','.join(str(e) for e in self.pontosPoligono)+')'

In [35]:
class LerXML:
    import xml.etree.ElementTree as Et
    
    def __init__(self,file) -> None:
        
        tree = self.Et.parse(file) #Gera uma árvore com o arquivo
        self.raiz = tree.getroot()          #Define a raiz


        #Percorre as raizes do arquivos, procurando todas as TAGS com esse nome
        #Nesse caso estamos percorrendo para pegar os valores da ViewPort
        for view in self.raiz.findall('viewport/vpmin'):
            vpmin = view.attrib #Atribui os atributos da TAG

        for view in self.raiz.findall('viewport/vpmax'):
            vpmax = view.attrib

        vpminX = float(vpmin.get('x')) #Pega o atributo X
        vpminY = float(vpmin.get('y')) #Pega o atributo Y
        vpmaxX = float(vpmax.get('x'))
        vpmaxY = float(vpmax.get('y'))


        #Vamos pegar agora os valores da Window
        for win in self.raiz.findall('window/wmin'):
            wmin = win.attrib

        for win in self.raiz.findall('window/wmax'):
            wmax = win.attrib

        wminX = float(wmin.get('x'))
        wminY = float(wmin.get('y'))
        wmaxX = float(wmax.get('x'))
        wmaxY = float(wmax.get('y'))


        self.janela = Window(wminX,wminY,wmaxX,wmaxY)            #Criando a instância da Window
        self.viewport = Viewport(vpminX,vpminY,vpmaxX,vpmaxY)    #Criando a instância da Viewport



        #Pegando os valores dos pontos
        self.pontos = [] #Criando um vetor de Pontos
        for pon in self.raiz.findall('ponto'):
            ponto = pon.attrib
            ponX = float(ponto.get('x'))
            ponY = float(ponto.get('y'))
            self.pontos.append(Ponto(ponX,ponY))                 #Adicionando os pontos ao valores

        #Pegando valores das Retas
        pontoRetas = [] #Criamos um vetor de pontos da reta
        self.retas = []      #Criamos um vetor com as retas

        for ret in self.raiz.findall('reta'):
            pontoRetas.clear()
            for ponto in ret:
                x = float(ponto.attrib.get('x'))
                y = float(ponto.attrib.get('y'))
                pontoRetas.append(Ponto(x,y)) #salvando os pontos da reta em um vetor
            reta = Reta(*pontoRetas)
            self.retas.append(reta) #salvadno a reta em um vetor

        
        #Pegar os pontos do Poligono
        pontosPoligono = [] #Fazemos o mesmo esquema das retas
        self.poligonos = []
        for poli in self.raiz.findall('poligono'):
            pontosPoligono.clear()
            for ponto in poli:
                x = float(ponto.attrib.get('x'))
                y = float(ponto.attrib.get('y'))
                pontosPoligono.append(Ponto(x,y))
            poligono = Poligono(*pontosPoligono)
            self.poligonos.append(poligono) #Mandamos um vetor de pontos para salvar o poli
    

In [36]:
class CriarXML:
    import xml.etree.ElementTree as Et

    def __init__(self,file,pontos,retas,poligonos) -> None:
        arq = self.Et.parse(file) 
        self.raiz = arq.getroot()

        i=0
        for self.ponto in self.raiz.findall('ponto'):
            self.ponto.set('x',str(pontos[i].x))    #Comando para trocar  os atributos 
            self.ponto.set('y',str(pontos[i].y))    #Atributo, valor modificar
            i+=1       #Conta a quantidade de pontos
        i=0
        for self.reta in self.raiz.findall('reta'):
            j=0         #reseta a cada reta, pois uma reta vai ser limitada a 2 pontos
            for h in self.reta.findall('ponto'):
                h.set('x',str(retas[i].pontosReta[j].x))
                h.set('y',str(retas[i].pontosReta[j].y))
                j+=1    #Conta a quantidade de pontos
            i+=1        #Conta a quantidade de retas
            
        i=0
        for self.poli in self.raiz.findall('poligono'):
            j=0         #reseta a cada reta, pois uma reta vai ser limitada a 2 pontos
            for h in self.poli.findall('ponto'):
                h.set('x',str(poligonos[i].pontosPoligono[j].x))
                h.set('y',str(poligonos[i].pontosPoligono[j].y))
                j+=1    #Conta a quantidade de pontos no poli
            i+=1        #Conta a quantidade de poli

        arq.write('NovoArquivo.xml')


In [37]:
class Transformada:

    def __init__(self,win,view) -> None:
        self.win = win
        self.view = view
        pass

    def transformarPonto(self,x,y) -> Ponto:
        self.pontoX = (((x-self.win.minX)/(self.win.maxX-self.win.minX))*(self.view.maxX-self.view.minX))
        self.pontoY = ((1-(y-self.win.minY)/(self.win.maxY-self.win.minY))*(self.view.maxY-self.view.minY))
        return Ponto(self.pontoX,self.pontoY)
    
    def transformarReta(self,pontos) -> Reta:
        pontosTrans = []
        for x in pontos:
            pontosTrans.append(self.transformarPonto(x.x,x.y))
        return Reta(*pontosTrans)
            
    
    def transformarPoli(self,pontos) -> Poligono:
        pontosTrans = []
        for x in pontos:
            pontosTrans.append(self.transformarPonto(x.x,x.y))
        return Poligono(*pontosTrans)
            
    def __str__(self) -> str:
        return (f"Viewport: {self.view}\nWindow: {self.win}")

In [38]:

if __name__ == '__main__':

    entrada = 'entrada.xml'
    file = LerXML(entrada)

    trans = Transformada(file.janela,file.viewport)
    print(trans)

    pontosTransformados = []
    retasTransformados = []
    poligonosTransformados = []

    for x in file.pontos:
        pontosTransformados.append(trans.transformarPonto(x.x,x.y))

    for retas in file.retas:
        retasTransformados.append(trans.transformarReta(retas.pontosReta))

    for poli in file.poligonos:
        poligonosTransformados.append(trans.transformarPoli(poli.pontosPoligono))

    CriarXML(entrada,pontosTransformados,retasTransformados,poligonosTransformados)

Viewport: (10.0,10.0,630.0,470.0)
Window: (0.0,0.0,10.0,10.0)
